# SQL

## Цели исследования

- Изучить данные таблиц;

- Посчитать, сколько книг вышло после 1 января 2000 года;

- Для каждой книги посчитать количество обзоров и среднюю оценку;

- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.


## Исследование таблиц

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 150

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### books

In [3]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### authors

In [4]:
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### publishers

In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### ratings

In [6]:
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### reviews

In [7]:
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


In [8]:
# какие ещё таблицы доступны в базе
display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,authors,praktikum_admin,None,True,False,True,False
4,public,second,praktikum_student,None,False,False,False,False
5,public,second_b,praktikum_student,None,False,False,False,False
6,public,publishers,praktikum_admin,None,True,False,True,False
7,public,author,praktikum_student,None,True,False,False,False
8,public,reviews,praktikum_admin,None,True,False,True,False
9,public,ratings,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,publishers,publisher_id,integer,NO
3,publishers,publisher,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


## SQL-запросы

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
query = '''SELECT 
                COUNT(*) AS number_of_books
                FROM
                    books
                WHERE publication_date >= '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,number_of_books
0,821


Таким образом, после 1 января 2000 года вышла 821 книга.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [10]:
query = ''' SELECT b.book_id,
            b.title,
            COUNT (DISTINCT rw.review_id) AS number_of_reviews, 
            ROUND (avg(r.rating), 2) AS average_rating
            FROM reviews AS rw
                RIGHT OUTER JOIN BOOKS AS b ON rw.book_id=b.book_id
                LEFT OUTER JOIN RATINGS AS r ON b.book_id = r.book_id
            GROUP BY b.book_id
            ORDER BY average_rating DESC'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,number_of_reviews,average_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #1-12),2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Meditation in Everyday Life,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman's Soul,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twenty-first Century,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


Таким образом, для каждой книги мы по id и заголовку выводим количество образов и среднюю оценку. Так, например, для книги A Woman of Substance (Emma Harte Saga #1), id которой 55, количество образов равно 2, а средняя оценка равна 5.0.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [11]:
query = ''' SELECT 
            p.publisher,
            COUNT(b.book_id) AS number_of_books
            FROM publishers AS p
                JOIN books AS b ON p.publisher_id=b.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY count(b.book_id) DESC
            LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,number_of_books
0,Penguin Books,42


Таким образом, издательством, которое выпустило наибольшее число книг толще 50 страниц, является Penguin Books	и оно выпустило 42 книги толще 50 страниц, что является наибольшим числом.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [19]:
query = '''SELECT 
                aut_avg.author,
                AVG(aut_avg.avg_rating) AS avg_rating
            FROM
                    (SELECT 
                        authors.author AS author
                       , AVG(ratings.rating) AS avg_rating
                    FROM books
                        LEFT JOIN authors ON books.author_id = authors.author_id
                        LEFT JOIN ratings ON books.book_id = ratings.book_id
                    GROUP BY 
                         books.book_id
                         , authors.author_id
                    HAVING
                        COUNT(ratings.rating_id) >=50 
                    ) AS aut_avg 
            GROUP BY    
                aut_avg.author
            ORDER BY
                avg_rating DESC
            LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Таким образом, автором с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками) является J.K. Rowling/Mary GrandPré, со средней оценкой  4.283844.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = ''' SELECT
            AVG(review.review_cnt) AS avg_review
            FROM
                (SELECT COUNT(review_id) AS review_cnt
            FROM 
                reviews
            WHERE
                username
            IN 
                (SELECT username
            FROM
                ratings
            GROUP BY
                username
            HAVING
            COUNT(ratings) > 48)
            GROUP BY 
                username) 
            AS
            review;'''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review
0,24.0


Таким образом, среднее количество обзоров от пользователей, которые поставили больше 48 оценок, 24.

## Общий вывод

Так, мы открыли файлы и изучили из подзапросы. Далее, с помощью SQL запросов нашли необходимые для нас данные: 

- После 1 января 2000 года вышла 821 книга.

- Для каждой книги мы по id и заголовку выводим количество образов и среднюю оценку. Так, например, для книги A Woman of Substance (Emma Harte Saga #1), id которой 55, количество образов равно 2, а средняя оценка равна 5.0.

- Издательством, которое выпустило наибольшее число книг толще 50 страниц, является Penguin Books и оно выпустило 42 книги толще 50 страниц, что является наибольшим числом.

- Автором с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками) является J.K. Rowling/Mary GrandPré, со средней оценкой 4.283844.

- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, 24.